
# Plotting Field Lines from sunkit-magex

``sunkit-pyvista`` can be used to plot field lines from ``sunkit-magex``.

This example requires the [streamtracer](https://streamtracer.readthedocs.io/en/stable/)_ package.


In [2]:
import warnings
warnings.filterwarnings('ignore')

# %matplotlib inline
# %matplotlib widget
# %matplotlib notebook
# import matplotlib
# matplotlib.use('nbAgg') # Agg, nbAgg, TkAgg, Qt5Agg, GTK3Agg

import pyvista as pv
pv.set_jupyter_backend('html') # static, client, server, trame, html
pv.start_xvfb()

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors

import astropy.units as u
from astropy.constants import R_sun
from astropy.coordinates import SkyCoord

import sunpy.map
from sunkit_magex import pfss
from sunkit_magex.pfss import tracing
from sunkit_magex.pfss.sample_data import get_gong_map
from sunpy.coordinates import frames

from sunkit_pyvista import SunpyPlotter
from sunkit_pyvista.sample import LOW_RES_AIA_193

In [5]:
plotter = SunpyPlotter()
plotter.plot_map(LOW_RES_AIA_193, clip_interval=[1, 99]*u.percent, assume_spherical_screen=False)
# plotter.plotter.add_axes(interactive=True, color='black')
plotter.plotter.add_axes(interactive=True, line_width=5, color='black',
                         x_color='red', y_color='green', z_color='blue')
plotter.plotter.set_background(color='Grey')
plotter.plot_solar_axis() # Add an arrow to show the solar rotation axis

# Set the camera coordinate to view the plot correctly
camera_coord = SkyCoord(
    0 * u.deg,
    0 * u.deg,
    6 * R_sun,
    frame=frames.HeliographicStonyhurst,
    obstime=LOW_RES_AIA_193.date)
plotter.set_camera_coordinate(camera_coord)
# plotter.plotter.view_xy() # view_xy, view_xz, view_yx, view_yz, view_zx, view_zy
# plotter.show(window_size=[400,400], cpos='yz', jupyter_backend='panel')
plotter.show()

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

In [11]:
from astropy.utils.data import download_file

filename = download_file(
    'http://jsoc.stanford.edu/data/hmi/synoptic/hmi.Synoptic_Mr.2191.fits', cache=True)
syn_map = sunpy.map.Map(filename)

plotter = SunpyPlotter()
plotter.plot_map(syn_map, cmap='hmimag', clip_interval=[1, 99]*u.percent, scalar_bar_args={'title': 'Radial Magnetic Field Strength'})
plotter.plotter.add_axes(interactive=True, line_width=5, color='black',
                         x_color='red', y_color='green', z_color='blue')
plotter.plotter.set_background(color='#54596d')
camera_coord = SkyCoord(
    0 * u.deg,
    0 * u.deg,
    6 * R_sun,
    frame=frames.HeliographicStonyhurst,
    obstime=syn_map.date)
plotter.set_camera_coordinate(camera_coord)
plotter.show()

INFO: Missing metadata for solar radius: assuming the standard radius of the photosphere. [sunpy.map.mapbase]


EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

In [10]:
from sunkit_magex.pfss.sample_data import get_gong_map

# load a gong_map from sunkit-magex
gong_fname = get_gong_map()
gong_map = sunpy.map.Map(gong_fname)

plotter = SunpyPlotter()
plotter.plot_map(gong_map, cmap='hmimag', clip_interval=[1, 99]*u.percent, scalar_bar_args={'title': 'Radial Magnetic Field Strength'})
plotter.plotter.add_axes(interactive=True, line_width=5, color='black',
                         x_color='red', y_color='green', z_color='blue')
plotter.plotter.set_background(color='#54596d')
camera_coord = SkyCoord(
    0 * u.deg,
    0 * u.deg,
    6 * R_sun,
    frame=frames.HeliographicStonyhurst,
    obstime=gong_map.date)
plotter.set_camera_coordinate(camera_coord)
plotter.show()

INFO: Missing metadata for solar radius: assuming the standard radius of the photosphere. [sunpy.map.mapbase]


EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

In [4]:
# [x for x in dir(plotter.plotter) if not x.startswith('_')]

We will be using an AIA 193 image from the sunpy sample data as the base image.



In [ ]:
# Start by creating a plotter
plotter = SunpyPlotter()

# Plot a map
plotter.plot_map(LOW_RES_AIA_193, clip_interval=[1, 99]*u.percent, assume_spherical_screen=False)
# Add an arrow to show the solar rotation axis
plotter.plot_solar_axis()
plotter.show()

ERROR:root:Cannot create GLX context.  Aborting.
libGL error: No matching fbConfigs or visuals found
libGL error: failed to load driver: swrast
2024-08-31 17:40:18.674 (   9.323s) [    7FE98BD9D740]vtkXOpenGLRenderWindow.:651    ERR| vtkXOpenGLRenderWindow (0x560e22a81b20): Cannot create GLX context.  Aborting.


We now need to do the magnetic field extrapolation using ``sunkit-magex``.



In [4]:
# We load a gong_map from sunkit-magex
gong_fname = get_gong_map()
gong_map = sunpy.map.Map(gong_fname)
# Now we plot the Gong Map to fill in the farside.
plotter.plot_map(gong_map, cmap="hmimag", clip_interval=[1, 99] * u.percent)

INFO: Missing metadata for solar radius: assuming the standard radius of the photosphere. [sunpy.map.mapbase]


In [5]:
# Create points spaced between lat={-90, 90} degrees
lat = np.linspace(-np.pi / 2, np.pi / 2, 32, endpoint=False)
# Create 32 points spaced between long={0, 360} degrees
lon = np.linspace(0, 2 * np.pi, 32, endpoint=False)
# Make a 2D grid from these 1D points
lat, lon = np.meshgrid(lat, lon, indexing="ij")
# Create lon, lat and radial coordinate values by using a sunkit-magex
# and trace them using tracer
lat, lon = lat.ravel() * u.rad, lon.ravel() * u.rad
# Define the number of grid points in rho and solar surface radius
nrho = 30
rss = 1.5
input_ = pfss.Input(gong_map, nrho, rss)
output_ = pfss.pfss(input_)
seeds = SkyCoord(lon, lat, 1.2 * R_sun, frame=gong_map.coordinate_frame)
tracer = tracing.FortranTracer()
field_lines = tracer.trace(seeds, output_)

/mnt/mnedal_data/anaconda3/envs/dias/lib/python3.10/site-packages/sunkit_magex/pfss/tracing.py:181: UserWarning: At least one field line ran out of steps during tracing.
You should probably increase max_steps (currently set to auto) and try again.
  warnings.warn(


We can also specify a color function while plotting the field lines.
This function takes a single field line, and returns a color either
in the form of a string, (r,g,b) or (r,g,b,a) tuple.
In this case we use a Matplotlib norm and colormap to return a tuple of RGBA values.



In [ ]:
def my_fline_color_func(field_line):
    norm = colors.LogNorm(vmin=1, vmax=1000)
    cmap = plt.get_cmap("viridis")
    return cmap(norm(np.abs(field_line.expansion_factor)))

# Plotting the field lines
plotter.plot_field_lines(field_lines, color_func=my_fline_color_func)

# Set the camera coordinate to view the plot correctly
camera_coord = SkyCoord(
    0 * u.deg,
    0 * u.deg,
    6 * R_sun,
    frame=frames.HeliographicStonyhurst,
    obstime=LOW_RES_AIA_193.date
)
plotter.set_camera_coordinate(camera_coord)
plotter.show()

ERROR:root:Cannot create GLX context.  Aborting.
libGL error: No matching fbConfigs or visuals found
libGL error: failed to load driver: swrast
2024-08-31 01:43:50.501 ( 180.601s) [    7FA772B29740]vtkXOpenGLRenderWindow.:651    ERR| vtkXOpenGLRenderWindow (0x564c2670f760): Cannot create GLX context.  Aborting.
